In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
from pathlib import Path

MYDRIVE = Path("/content/drive/MyDrive")

import os
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

!pip -q install transformers accelerate
import torch
from transformers import AutoImageProcessor, AutoModel

BT_ROOT = Path("/content/drive/MyDrive/Dataset/BT")
TRAIN_DIR = BT_ROOT / "Training"
TEST_DIR  = BT_ROOT / "Testing"

print("Train dir exists?", TRAIN_DIR.exists(), TRAIN_DIR)
print("Test  dir exists?", TEST_DIR.exists(), TEST_DIR)

valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.webp'}
valid_classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

def collect_images(root_dir: Path):
    files, labels = [], []
    for cls in valid_classes:
        cls_dir = root_dir / cls
        if not cls_dir.exists():
            print(f"⚠️ Missing folder: {cls_dir}")
            continue
        for fp in cls_dir.rglob("*"):
            if fp.is_file() and fp.suffix.lower() in valid_extensions:
                files.append(str(fp))
                labels.append(cls)
    return files, labels

if TRAIN_DIR.exists():
    X_train_paths, y_train = collect_images(TRAIN_DIR)
    X_test_paths,  y_test  = collect_images(TEST_DIR)
else:
    X_all, y_all = collect_images(TEST_DIR)
    from sklearn.model_selection import train_test_split
    X_train_paths, X_test_paths, y_train, y_test = train_test_split(
        np.array(X_all), np.array(y_all),
        test_size=0.2, random_state=42, stratify=np.array(y_all)
    )

print("\n✅ Counts:")
print("Train images:", len(X_train_paths))
print("Test  images:", len(X_test_paths))

if len(X_train_paths) == 0 or len(X_test_paths) == 0:
    raise ValueError("❌ No images found in train/test. Check folders & image extensions.")

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "google/vit-base-patch16-224-in21k"

processor = AutoImageProcessor.from_pretrained(model_name)
vit_model = AutoModel.from_pretrained(model_name).to(device)
vit_model.eval()

def load_image(path):
    return Image.open(path).convert("RGB")

@torch.no_grad()
def extract_embeddings(image_paths, batch_size=16):
    feats = []
    for i in tqdm(range(0, len(image_paths), batch_size), desc="Extracting ViT Embeddings"):
        batch_paths = image_paths[i:i+batch_size]
        images = [load_image(p) for p in batch_paths]
        inputs = processor(images=images, return_tensors="pt").to(device)
        outputs = vit_model(**inputs)
        cls_emb = outputs.last_hidden_state[:, 0, :].detach().cpu().numpy()  # CLS token
        feats.append(cls_emb)
    return np.vstack(feats)

X_train = extract_embeddings(X_train_paths, batch_size=16)
X_test  = extract_embeddings(X_test_paths,  batch_size=16)

print("Train features:", X_train.shape, "Test features:", X_test.shape)

models = {
    "1) Logistic Regression": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=5000))
    ]),
    "2) SVM (RBF)": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf", C=10, gamma="scale"))
    ]),
    "3) Random Forest": RandomForestClassifier(
        n_estimators=300, random_state=42, n_jobs=-1
    ),
    "4) KNN": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=7))
    ]),
    "5) Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

results = []

for name, clf in models.items():
    print("\n" + "="*70)
    print("Model:", name)

    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)

    acc = accuracy_score(y_test, preds)
    results.append((name, acc))

    print("✅ Accuracy:", round(acc, 4))
    print("\nClassification Report:")
    print(classification_report(y_test, preds, digits=4))

    print("Confusion Matrix (order: glioma, meningioma, notumor, pituitary):")
    print(confusion_matrix(y_test, preds, labels=valid_classes))

res_df = pd.DataFrame(results, columns=["Model", "Accuracy"]).sort_values("Accuracy", ascending=False)
print("\n" + "="*70)
print("🏁 Final Accuracy Comparison:")
print(res_df.to_string(index=False))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train dir exists? True /content/drive/MyDrive/Dataset/BT/Training
Test  dir exists? True /content/drive/MyDrive/Dataset/BT/Testing

✅ Counts:
Train images: 5712
Test  images: 1311


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Extracting ViT Embeddings: 100%|██████████| 82/82 [18:37<00:00, 13.63s/it]


Train features: (5712, 768) Test features: (1311, 768)

Model: 1) Logistic Regression
✅ Accuracy: 0.9748

Classification Report:
              precision    recall  f1-score   support

      glioma     0.9928    0.9233    0.9568       300
  meningioma     0.9315    0.9771    0.9537       306
     notumor     0.9951    1.0000    0.9975       405
   pituitary     0.9770    0.9900    0.9834       300

    accuracy                         0.9748      1311
   macro avg     0.9741    0.9726    0.9729      1311
weighted avg     0.9756    0.9748    0.9748      1311

Confusion Matrix (order: glioma, meningioma, notumor, pituitary):
[[277  19   1   3]
 [  2 299   1   4]
 [  0   0 405   0]
 [  0   3   0 297]]

Model: 2) SVM (RBF)
✅ Accuracy: 0.9817

Classification Report:
              precision    recall  f1-score   support

      glioma     0.9861    0.9433    0.9642       300
  meningioma     0.9469    0.9902    0.9681       306
     notumor     1.0000    1.0000    1.0000       405
   pituitary